## For an unsolvable system of linear equations, Ax = b, there exists a best solution, x_hat, that is found by 
## solving the equation p = Ax_hat, where p is the vector resulting from the projection of b onto the column 
## space of A. Since e = b-p is orthogonal to the column space of A (i.e., belongs to the left-nullspace of A), 
## it follows that A'(b-Ax_hat) = 0 and A'Ax_hat = A'b, allowing us to solve for x_hat.

In [50]:
import pandas as pd
import numpy as np

In [51]:
seed = 311
rng = np.random.default_rng(seed) 

In [52]:
df = pd.DataFrame(rng.integers(0,100,size=(100, 4)), columns=['x1','x2','x3','x4'])
df

,x1,x2,x3,x4
0,34,15,23,56
1,8,64,4,70
2,46,13,6,79
3,60,98,43,8
4,2,18,33,8
...,...,...,...,...
95,0,68,99,27
96,33,59,21,33
97,7,79,49,84
98,6,57,0,43


In [53]:
correlations = rng.uniform(low=-1.0, high=1.0, size=4)
correlations

array([-0.16206972, -0.84142446,  0.76057477, -0.11313913])

In [54]:
# https://stackoverflow.com/questions/42902938/create-correlated-pandas-series

from scipy.stats import pearsonr
from scipy.optimize import minimize

# data = pd.DataFrame({'Country A': [10, 11, 10, 9]})

# data['Country B'] = minimize(lambda x: abs(0.8 - pearsonr(data['Country A'], x)[0]), 
#                              np.random.rand(len(data['Country A']))).x

# df['y1'] = (minimize(lambda x: abs(correlations[0] - pearsonr(df['x1'], x)[0]),
#                                   rng.random(len(df))).x) * 100

# df['y2'] = (minimize(lambda x: abs(correlations[1] - pearsonr(df['x2'], x)[0]),
#                                   rng.random(len(df))).x) * 100

# df['y3'] = (minimize(lambda x: abs(correlations[2] - pearsonr(df['x3'], x)[0]),
#                                   rng.random(len(df))).x) * 100

# df['y4'] = (minimize(lambda x: abs(correlations[3] - pearsonr(df['x4'], x)[0]),
#                                   rng.random(len(df))).x) * 100

df['y'] = (minimize(lambda x: abs(
                                 (correlations[0] - pearsonr(df['x1'], x)[0])
                                  + (correlations[1] - pearsonr(df['x2'], x)[0])
                                  + (correlations[2] - pearsonr(df['x3'], x)[0])
                                  + (correlations[3] - pearsonr(df['x4'], x)[0])
                                 ), rng.random(len(df))).x) * 100

display(df)

,x1,x2,x3,x4,y
0,34,15,23,56,37.137474
1,8,64,4,70,81.414166
2,46,13,6,79,60.273728
3,60,98,43,8,84.242199
4,2,18,33,8,103.286051
...,...,...,...,...,...
95,0,68,99,27,41.289700
96,33,59,21,33,10.294394
97,7,79,49,84,82.080967
98,6,57,0,43,98.852285


In [29]:
# df['y'] = np.round((df['y1'] + df['y2'] +df['y3'] + df['y4']), 0)
# df = df.drop(columns=['y1', 'y2', 'y3', 'y4'])
# df

In [55]:
np.set_printoptions(precision=8, threshold=1000, edgeitems=3, linewidth=75, suppress=True, nanstr='nan', infstr='inf')

In [57]:
class Linear_Regression:
    
    def __init__(self):
        print('linear regression object instantiated')
    
    def linear_regression(self, df: pd.DataFrame, label: str):
        # Create coefficient matrix and dependent variable vector
        b = df[label].to_numpy()
        if df.iloc[:,0].sum() != len(df): # if there's no intercept column
            df.insert(loc = 0, column = 'x0', value = np.ones(len(df)))
        A = df.drop(label, axis=1).to_numpy()
        # Solve A_T_A x = A_T b for x
        A_T = np.transpose(A)
        A_T_A = np.matmul(A_T, A)
        A_T_b = np.matmul(A_T, b)
        x = np.linalg.solve(A_T_A, A_T_b)
        # Solve for p and compute sse
        p = np.matmul(A, x)
        e = b-p
        train_sse = (np.linalg.norm(e))**2.
        # Return coefficients for best solution and sse
        return x, train_sse

    def fit(self, df: pd.DataFrame, label: str):
        coefficients, sse = linear_regression(df,label)
        intercept = coefficients[0]
        coefficients = coefficients[1:]
        return intercept, coefficients
        

In [31]:
def linear_regression(df: pd.DataFrame, label: str):

    # Create coefficient matrix and dependent variable vector

    b = df[label].to_numpy()

    if df.iloc[:,0].sum() != len(df): # if there's no intercept column
        df.insert(loc = 0, column = 'x0', value = np.ones(len(df)))

    A = df.drop(label, axis=1).to_numpy()

    # Solve A_T_A x = A_T b for x

    A_T = np.transpose(A)

    A_T_A = np.matmul(A_T, A)

    A_T_b = np.matmul(A_T, b)

    x = np.linalg.solve(A_T_A, A_T_b)

    # Solve for p and compute sse

    p = np.matmul(A, x)

    e = b-p
    
    train_sse = (np.linalg.norm(e))**2.

    # Return coefficients for best solution and sse

    return x, train_sse

In [45]:
coefficients, sse = linear_regression(df,'y')
intercept = coefficients[0]
coefficients = coefficients[1:]
print(intercept)
print(coefficients)

68.91167632039958
[-0.17432897 -0.1168181  -0.04999463 -0.06386287]


In [46]:
correlations

array([-0.16206972, -0.84142446,  0.76057477, -0.11313913])

In [34]:
'''
>>> import numpy as np
>>> from sklearn.linear_model import LinearRegression
>>> X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
>>> # y = 1 * x_0 + 2 * x_1 + 3
>>> y = np.dot(X, np.array([1, 2])) + 3
>>> reg = LinearRegression().fit(X, y)
>>> reg.score(X, y)
1.0
>>> reg.coef_
array([1., 2.])
>>> reg.intercept_
3.0...
>>> reg.predict(np.array([[3, 5]]))
array([16.])
'''
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(df[['x1','x2','x3','x4']].to_numpy(), df['y'].to_numpy())
print(reg.intercept_)
reg.coef_

68.91167632039965


array([-0.17432897, -0.1168181 , -0.04999463, -0.06386287])

In [ ]:
(array([68.91167632, -0.17432897, -0.1168181 , -0.04999463, -0.06386287]),
 95844.61724311876)

In [49]:
print('Intercepts')
print(f'Scikit-Learn:      {reg.intercept_}')
print(f'Linear_Regression: {intercept}')
print('')
print('Coefficients')
print(f'Scikit-Learn:      {reg.coef_}')
print(f'Linear_Regression: {coefficients}')

Intercepts
Scikit-Learn:      68.91167632039965
Linear_Regression: 68.91167632039958

Coefficients
Scikit-Learn:      [-0.17432897 -0.1168181  -0.04999463 -0.06386287]
Linear_Regression: [-0.17432897 -0.1168181  -0.04999463 -0.06386287]


In [60]:
lin_reg = Linear_Regression()
lin_reg.fit(df,'y')

linear regression object instantiated


(68.91167632039958,
 array([-0.17432897, -0.1168181 , -0.04999463, -0.06386287]))